In [ ]:
# 1. Create '房地總價元' column
# 2. Add 格局比例 4 columns: 房、廳、衛、隔間
# 3. One-hot encoding for '是否包含車位' by extracting the number after '車位'

In [17]:
import pandas as pd

df = pd.read_csv('./updated_geocode_1016.csv')

pd.set_option('display.max_columns', None)

df.head(1)


,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,交易筆棟數,移轉層次,總樓層數,建物型態,主要用途,主要建材,建築完成年月,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,總價元,單價元平方公尺,車位類別,車位移轉總面積平方公尺,車位總價元,備註,編號,主建物面積,附屬建物面積,陽台面積,電梯,移轉編號,建案名稱,棟及號,解約情形,緯度,經度
0,西港區,房地(土地+建物),臺南市西港區溪埔寮３３之３９號,82.39,NaN,鄉村區,乙種建築用地,2022-03-23,土地3建物1車位0,全,四層,透天厝,住家用,鋼筋混凝土造,1120802.0,155.06,4,2,3,有,無,10600000,68361.0,NaN,0.0,0,預售屋、或土地及建物分件登記案件；,RPVOMLKKMHJGFGD38DA,148.74,0.0,6.32,無,NaN,NaN,NaN,NaN,23.084845,120.186315


In [18]:
# 1. Create '房地總價元' column
df['房地總價元'] = df['總價元'] - df['車位總價元']

In [19]:
# 2. Add 格局比例 4 columns: 房、廳、衛、隔間

# Replace '有' with 1 and '無' with 0 in '建物現況格局-隔間'
df['建物現況格局-隔間-轉數字'] = df['建物現況格局-隔間'].replace({'有': 1, '無': 0})

# Calculate the total number of rooms
total_rooms = df[['建物現況格局-房', '建物現況格局-廳', '建物現況格局-衛', '建物現況格局-隔間-轉數字']].sum(axis=1)

# Calculate proportions
df['格局比例_房'] = df['建物現況格局-房'] / total_rooms
df['格局比例_廳'] = df['建物現況格局-廳'] / total_rooms
df['格局比例_衛'] = df['建物現況格局-衛'] / total_rooms
df['格局比例_隔間'] = df['建物現況格局-隔間-轉數字'] / total_rooms

# Drop the temporary '建物現況格局-隔間-轉數字' column
df.drop(columns=['建物現況格局-隔間-轉數字'], inplace=True)

# Replace NaN with 0 in the proportion columns
df[['格局比例_房', '格局比例_廳', '格局比例_衛', '格局比例_隔間']] = df[['格局比例_房', '格局比例_廳', '格局比例_衛', '格局比例_隔間']].fillna(0)

# Replace values equal to 1 in '格局比例_廳' and '格局比例_衛' with 0
df['格局比例_廳'] = df['格局比例_廳'].replace(1, 0)
df['格局比例_衛'] = df['格局比例_衛'].replace(1, 0)

/var/folders/6g/zb_yk76n3gx45563jpl8swnw0000gn/T/ipykernel_92963/1911475016.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['建物現況格局-隔間-轉數字'] = df['建物現況格局-隔間'].replace({'有': 1, '無': 0})


In [20]:
# 3. One-hot encoding for '是否包含車位' by extracting the number after '車位'
df['是否包含車位'] = df['交易筆棟數'].str.extract(r'車位(\d+)').fillna(0).astype(int)
df['是否包含車位'] = df['是否包含車位'].apply(lambda x: 1 if x > 0 else 0)

In [21]:
# Check NaN values
columns_to_check = ['房地總價元', '格局比例_房', '格局比例_廳', '格局比例_衛', '格局比例_隔間', '是否包含車位']

nan_rows = df[df[columns_to_check].isnull().any(axis=1)]

nan_rows.shape

(0, 44)

In [22]:
df.to_csv('geocode_and_onehot_yanru_1016.csv', index=False)